# Generatory

W trakcie wywoływania funkcje otrzymują prywatną przestrzeń nazw - tworzone są w niej zmienne lokalne, które po osiągnięciu instrukcji $\texttt{return}$ i zwróceniu odpowiedniego wyniku są usuwane z pamięci komputera.

In [ ]:
def function(n):
    res = 0
    for i in range(n+1):
        x = i**2
        res += x
    return res

In [ ]:
function(2)

In [ ]:
x

Kolejne wywołania funkcji korzystają z "surowego", zadanego na początku zbioru danych i wyliczają wartości funkcji na nowo. Co jednak, jeżeli byśmy chcieli nie usuwać zmiennych lokalnych danej funkcji i wykorzystać je później, przy kolejnym jej wywołaniu?

Wtedy musimy skorzystać z generatorów.

### Dlaczego generatory?

Przede wszystkim generatory ze względu na swoją konstrukcję pozwalają na zaoszczędzenie pamięci komputera. Rozpatrzmy prosty przykład, chcemy policzyć sumę sześcianów wszystkich liczb naturalnych od $1$ do $n$, gdzie $n$ przyjmuje bardzo dużą wartość, np. $1000000$. Zapisując to jako pythonowską funkcję otrzymamy następujący kod:

In [ ]:
def numbers3 (n):
    i = 1
    numbers = []
    while i <= n:
        numbers.append(i**3)
        i += 1
    return numbers

sum(numbers3(1000000))

Tak długa i zawierająca tak duże wartości lista będzie nieefektywna - będzie zajmować mnóstwo przestrzeni na dysku i w znacznym stopniu spowolni działanie kodu (w skrajnych przypadkach może wręcz uniemożliwić jego wykonanie). 

W takim wypadku przewaga generatora nad funkcją wynika z dwóch jego cech. Po pierwsze, generatory generują kod na żądanie - dzięki temu możliwe jest wykorzystywanie wygenerowanych elementów na bieżąco, bez konieczności czekania aż generator skończy pracować. Po drugie, generatory przechowują stany pomiędzy wywołaniami, a nie wszystkie otrzymane przy jego wykorzystaniu wartości.

In [ ]:
def gen_numbers3 (n):
    i = 1
    while i <= n:
        yield i**3
        i +=1 
        
sum(gen_numbers3(1000000))

In [ ]:
gen_numbers3(100000000000000)

Generatory definiuje się bardzo podobnie do funkcji, różnica polega na tym, że zamiast słowa kluczowego $\texttt{return}$ wykorzystujem słowo kluczowe $\texttt{yield}$. Można stworzyć generator za pomocą składni analogicznej do listy składanej. Wtedy zamiast nawiasów kwadratowych korzystamy ze zwykłych, okrągłych nawiasów.

In [ ]:
generator = (i**2 for i in range(20))
generator

Aby wywołać kolejne generowane elementy możemy skorzystać z funkcji $\texttt{next}$:

In [ ]:
next(generator)

Wyniki można też opakować w listę:

In [ ]:
generator = (i**2 for i in range(10))
list(generator)

In [ ]:
generator = (i**2 for i in range(20))

In [ ]:
for i in generator:
    print (i)
    if i > 55:
        break

In [ ]:
list(generator)

In [ ]:
next(generator)

### Współprogramy


Typowym sposobem pisania programów komputerowych jest modularyzacja złożonego zadania obliczeniowego czyli jego podział na mniejsze, logicznie wyodrębnione jednostki. Jednostki takie przybierają w zależności od wybranego języka programowania formę funkcji, procedur albo klas.
Po dekompozycji zadania obliczeniowego na moduły określa się sposób ich współpracy. Najczęściej koordynacje pracy modułów powierza się głównej funkcji sterującej (zwyczajowo nazywanej `main`). Funkcja sterująca okresla nastepstwo wykonania i zasady współpracy poszczególnych modułów przy użyciu (poznanych już) instrukcji warunkowych, petli itd. W takim podejściu moduły przyjęło się nazywać **podprogramami** (ang. *subroutine*).

Istnieje alternatywny sposób koordynacji pracy podzadań obliczeniowych bez użycia funkcji sterującej, w którym przebieg przetwarzania wynika wyłącznie z komunikacji między modułami. Przepływ sterowania w tym podejściu określa się przez łączenie zadań w łańcuchy (potoki). Takie podejście to nazywa się też czasami sterowaniem przez zdarzenia (ang. *event driven*). W takim podejściu moduły przyjęło się nazywać **współprogramami** (ang. *coroutine*). Jak się przekonamy podejście oparte o  współprogramy doskonale pasuje do modelowania złożonych systemów kolejkowych.

Sposób działania współprogramów jest bardzo podobny do sposobu działania generatorów (właściwie są one specjalnym typem współprogramów). Co więcej, w Pythonie tworzy się je w dokładnie ten sam sposób.

#### Wzorzec produkuj-filtruj-konsumuj

Typowym sposobem wykorzystania potoku przetwarzania jest wzorzec _'producent-konsument'_, w którym współprogram pełni jedną z trzech ról:
- **producenta**, który generuje dane wejściowe dla potoku korzystając z medody `send()` 
- **filtra**, który  przetwarza dane, nastepnie przesyła je do przetwarzania na kolejnych etapach potoku korzystając z metod `yield()` i `send()`
- **konsumenta**, który dokonuje końcowego przetwarzania danych otrzymanych z wykorzystaniem metody `yield()` i przekazuje je na wyjście potoku

Wzorzec produkuj-filtruj-konsumuj doskonale nadaje się do tworzenia prostych symulacji opartych o czas dyskretny. Stwórzmy więc uproszczoną wersję (1 maszyna i 1 zestaw narzędzi, brak czasu dostawy) omawianego modelu:

In [ ]:
import numpy as np

REPAIR_TIME = 15
WORKING_TIME = 75

# konsument - odbiorca zgłoszeń
def consumer():
    broken_time = 0
    end_time = 0
    while True:
        broken_time = yield broken_time, end_time
        service_time = np.random.gamma(3, REPAIR_TIME/3)
        end_time = broken_time + service_time
        
# producent - generator zgłoszeń        
def producer(horizon, server):
    broken_time = np.random.exponential(WORKING_TIME)
    print('maszyna zepsula sie o %d' % broken_time)
    while broken_time <= horizon:
        result = server.send(broken_time)
        print('koniec naprawy o %d' % result[1])
        broken_time = result[1] + np.random.exponential(WORKING_TIME)
        print('maszyna zepsula sie o %d' % broken_time)
        
# tworzymy i inicjujemy serwer        
server = consumer()

next(server)

#uruchamiamy symulacje
producer(30*24*60, server)

Oczywiście za pomocą schematu produkuj-filtruj-konsumuj możliwe jest stworzenie bardziej wyszukanych symulacji - uwzględnienie większej ilości maszyn i zestawów narzędziowych. Wymaga to jednak stworzenia odpowiedniego współprogramu-filtra, który ustali odpowiednią kolejność wywołań. Najprostszą metodą na zrobienie tego jest skorzystanie z biblioteki [simpy](https://simpy.readthedocs.io/en/latest/).